In [53]:
from datasets import load_dataset, load_metric, list_metrics
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np 

In [54]:
dataset = load_dataset("data/depressive_speech")
#dataset
dataset['train']

Using custom data configuration depressive_speech-96701654af5ae1e8
Reusing dataset csv (/Users/haydenprescott/.cache/huggingface/datasets/csv/depressive_speech-96701654af5ae1e8/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58)


  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['id', 'text', 'label', 'source_data'],
    num_rows: 10843
})

In [55]:
model_name = "bert-base-cased"

In [56]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [57]:
def tokenize(samples):
    return tokenizer(samples['text'], padding="max_length", truncation=True)

In [58]:
dataset_tokens = dataset.map(tokenize, batched=True)
label_count = np.unique(np.array(dataset_tokens['train']['label'])).size
label_count

  0%|          | 0/11 [00:00<?, ?ba/s]

2

In [59]:
shuffle = dataset_tokens['train'].shuffle(seed=42)

train_count = int(shuffle.num_rows * 0.9) 

dataset_train = shuffle.select(range(0, train_count))
dataset_eval = shuffle.select(range(train_count, shuffle.num_rows))
print("Eval:", dataset_eval, "\nTrain:", dataset_train)

Eval: Dataset({
    features: ['id', 'text', 'label', 'source_data', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1085
}) 
Train: Dataset({
    features: ['id', 'text', 'label', 'source_data', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 9758
})


In [60]:
base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=label_count)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [61]:
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [62]:
acc_metric = load_metric("accuracy")

In [63]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return acc_metric.compute(predictions=predictions, references=labels)

In [64]:
trainer = Trainer(
    model=base_model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_eval,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: source_data, text, id. If source_data, text, id are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/Users/haydenprescott/opt/anaconda3/envs/hf-transformers/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 9758
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3660
